<a href="https://colab.research.google.com/github/hchung04/StrikeBallMLClassifier/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This code connects google Colab with your google drive allowing you to use stuff

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import datasets
from sklearn import metrics
from pandas import DataFrame
import statistics
from scipy.stats import norm
from scipy.stats import f_oneway
import csv
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

df = pd.read_csv('/content/drive/Shareddrives/Hera/Datasets/merged*.csv') #df = dataframe

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
  # KNeighborsClassifier(3),
  # SVC(kernel="linear", C=0.025),
  # SVC(gamma=2, C=1),
  # GaussianProcessClassifier(1.0 * RBF(1.0)),
  # DecisionTreeClassifier(max_depth=5),
  # RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
  # MLPClassifier(alpha=1, max_iter=1000),
  # AdaBoostClassifier(),
  # GaussianNB(),
  # QuadraticDiscriminantAnalysis(),
  # LogisticRegression(multi_class='multinomial', solver='lbfgs')
]

<ipython-input-2-b274663b9e18>:28: DtypeWarning: Columns (27,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/Shareddrives/Hera/Datasets/merged*.csv') #df = dataframe


In [3]:
# Classifying pitches as balls or strikes and removes pitches that are neither
  # starting only with TaggedPitchType + PitchCall, PlateLocSide, PlateLocHeight
def classify(x):
  if x == "StrikeCalled":
    return 'Strike'
  elif x == "BallCalled" or x == "HitByPitch" or x =="BallinDirt":
    return 'Ball'
  else:
    return None

df['Classification'] = df['PitchCall'].apply(classify)
# Removes rows that aren't classified as a ball or strike
df1 = df[["Classification", "PlateLocSide", "PlateLocHeight"]].dropna()

# excluded 'Tilt' and 'MaxHeight' for the time being
df2 = df[["Classification", "RelSpeed",	"VertRelAngle",	"HorzRelAngle",
          "SpinRate",	"SpinAxis",	"RelHeight", "RelSide", "Extension",
          "VertBreak", "InducedVertBreak", "HorzBreak",	"PlateLocHeight",
          "PlateLocSide",	"ZoneSpeed", "VertApprAngle",	"HorzApprAngle",
          "ZoneTime",	"pfxx",	"pfxz",	"x0",	"y0",	"z0",	"vx0", "vy0",	"vz0",
          "ax0", "ay0", "az0", "EffectiveVelo",	"SpeedDrop",
          "PitchTrajectoryXc0",	"PitchTrajectoryXc1",	"PitchTrajectoryXc2",
          "PitchTrajectoryYc0", "PitchTrajectoryYc1",	"PitchTrajectoryYc2",
          "PitchTrajectoryZc0",	"PitchTrajectoryZc1",
          "PitchTrajectoryZc2"]].dropna()

In [4]:
# Finds the correlation between data values and the categorization as ball/strike
# H0: Classification and [ColumnName] are NOT correlated
for col in df2.columns:
  if col != 'Classification':
    # Finds out the PlateLocSide data for each Classification as a list
    CategoryGroupLists=df2.groupby('Classification')[col].apply(list)
    # Performing the ANOVA test
    AnovaResults = f_oneway(*CategoryGroupLists)
    if AnovaResults[1] >= 0.05:
      df2 = df2.drop(columns=[col])

In [5]:
# df2['Classification'].value_counts()

msk = df2['Classification'] == 'Strike'
num_to_oversample = len(df2) - 2*msk.sum()
X_positive_oversample = df2[msk].sample(n=num_to_oversample, replace=True, random_state=888)
X_train_oversample = pd.concat([df2, X_positive_oversample])

# X_train_oversample['Classification'].value_counts()

X = X_train_oversample.drop(columns=['Classification'])
Y = X_train_oversample['Classification']
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [6]:
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.pipeline import make_pipeline
# from sklearn.svm import LinearSVC

# # Anova SVM
# anova_filter = SelectKBest(f_classif, k="all")
# clf = LinearSVC(dual=True)
# anova_svm = make_pipeline(anova_filter, clf)
# anova_svm.fit(X_train, y_train)

# K Nearest Neighbors
# Scale the features using StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(X_train, y_train)

# Linear SVM, Logistic Regression - uh did not work
# classifier = SVC(kernel='linear', random_state=0)
# classifier.fit(X_train, y_train)

#Logistic Regression
# from sklearn.linear_model import LogisticRegression
# logreg = LogisticRegression()
# logreg.fit(X_train, y_train)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [7]:
# shows classification of true positive, false positives, false negatives, and true negatives
with open('/content/drive/Shareddrives/Hera/output.txt', "a") as f:
  from sklearn.metrics import classification_report
  target_names = ['Ball', 'Strike']
  y_pred = model.predict(X_test)
  # print("Decision Tree (default)", file =f)
  print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

        Ball       0.96      0.92      0.94    115891
      Strike       0.92      0.96      0.94    116023

    accuracy                           0.94    231914
   macro avg       0.94      0.94      0.94    231914
weighted avg       0.94      0.94      0.94    231914

